In [1]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-cp36-none-any.whl size=19879 sha256=1a0d6b002215004c7b03909a019dcc294f60024319fb491fa8b361f26629f1d7
  Stored in directory: /root/.cache/pip/wheels/5f/bb/4e/f6488433d53bc0684673d6845e5bf11a25240577c8151c140e
Successfully built vecstack


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

import pandas as pd
import numpy as np

#changing display settings for columns, rows and width
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#for preprocessing
from sklearn.preprocessing import LabelEncoder
from scipy.stats import mode
from sklearn.preprocessing import MinMaxScaler

#for point prediction
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import SCORERS
import math

#for stacking
from vecstack import stacking


import warnings
warnings.filterwarnings("ignore")




Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
#importing data
trainfile = r'/gdrive/My Drive/Assignments/TeamAssignment1/trainupdated.csv'
testfile = r'/gdrive/My Drive/Assignments/TeamAssignment1/testupdated.csv'

#copying into a DF
train_data = pd.read_csv(trainfile)
test_data = pd.read_csv(testfile)

#checking data shape
print(train_data.shape)
print(test_data.shape)

(1460, 81)
(1459, 80)


In [4]:
#Checking data columns 
print(train_data.columns)

#using python to understand columns
print(train_data.describe(include ='all'))
print("\n\n\n\n")
print(test_data.describe(include ='all'))

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual', 'GarageCond', 'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea',
  

In [5]:
#extracting target column before checking for missing values.
Y_train = train_data["SalePrice"]
#dropping target column from train_data
train_data = train_data.drop(["SalePrice"], axis = 1)

print(train_data.shape)


(1460, 80)


In [59]:
#Dropping columns with lots of missing values
#combining train and test data for this purpose to ensure consistency.
combined_data = pd.concat([train_data, test_data], keys = [0,1])

#Looking at column null values count.
colNullValDF = combined_data.isnull().sum().to_frame()
print(colNullValDF.T)
colNullValDF.to_csv(r'/gdrive/My Drive/Assignments/TeamAssignment1/ColNullVal.csv')

   Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  OverallQual  OverallCond  YearBuilt  YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  Functional  Fireplaces  FireplaceQu  GarageType  GarageYrBlt  GarageFinish  GarageCars  GarageArea  GarageQual  GarageCond  PavedDrive  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  PoolQC  Fence  MiscFeature  MiscVal  MoSold  YrSold  SaleType  SaleCondition
0   0           0         4          486        0       0      0         

In [7]:
#Establishing threshold to drop the columns
threshold = int(0.20*(combined_data.shape[0]))
print(threshold)

#drop columns that have less than threshold number of non_nans
TestTrainWithColDrop = combined_data.dropna(thresh= threshold, axis = 1)
print(TestTrainWithColDrop.shape)
print("\n\nMissing Values:")
#reverifying columns with missing values
print(TestTrainWithColDrop.isnull().sum().to_frame().T)
print("\n\nValue Counts:")
#understanding column datatypes
print(TestTrainWithColDrop.dtypes.value_counts())
print("\n\nColumn Data Types determined by python:")
print(pd.DataFrame(TestTrainWithColDrop.dtypes).T)
pd.DataFrame(TestTrainWithColDrop.dtypes).to_csv(r'/gdrive/My Drive/Assignments/TeamAssignment1/columnDataType.csv')

583
(2919, 80)


Missing Values:
   Id  MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  OverallQual  OverallCond  YearBuilt  YearRemodAdd  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  MasVnrArea  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  Heating  HeatingQC  CentralAir  Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  Functional  Fireplaces  FireplaceQu  GarageType  GarageYrBlt  GarageFinish  GarageCars  GarageArea  GarageQual  GarageCond  PavedDrive  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  PoolQC  Fence  MiscFeature  MiscVal  MoSold  YrSold  SaleType  SaleCondition
0   0           0         4          486

In [8]:
#filling the missing values for nan in numeric columns
"""
of the columns identified as integers MSSubClass, OveralQual and 
OverallCond are not integer type. They are actually categorical.
Therefore its important to first remove them before numeric columns filling
"""
print(TestTrainWithColDrop.shape)
num_combined_data = TestTrainWithColDrop.select_dtypes(include=['int','float64'])
num_combined_data = num_combined_data.drop(columns = ['MSSubClass','OverallQual','OverallCond'], axis = 1)
ActuallyCategorical = pd.concat([TestTrainWithColDrop['MSSubClass'],TestTrainWithColDrop['OverallQual'],TestTrainWithColDrop['OverallCond']], axis = 1)
print(ActuallyCategorical.head())
print(num_combined_data.shape)


(2919, 80)
     MSSubClass  OverallQual  OverallCond
0 0          60            7            5
  1          20            6            8
  2          60            7            5
  3          70            7            5
  4          60            8            5
(2919, 34)


In [12]:
#Now giving mean values for nan in numeric columns
numeric = num_combined_data.select_dtypes(include=['int','float64']).columns
for num in numeric:
  num_combined_data[num] = num_combined_data[num].fillna(num_combined_data[num].median())

#checking missing values in num_combined_data
num_combined_dataDF = num_combined_data.isnull().sum().to_frame()
print(num_combined_dataDF.T)

   Id  LotFrontage  LotArea  YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold
0   0            0        0          0             0           0           0           0          0            0         0         0             0          0             0             0         0         0             0             0             0           0            0           0           0           0            0              0          0            0         0        0       0       0


In [21]:
#Filling missing values for categorical columns
#Obtaining categorical columns identified by python
catColObtained = TestTrainWithColDrop.select_dtypes(include = ['object'])
#First combining the previously excluded cat variables labeled as numeric by python
cat_combined_data = pd.concat([ActuallyCategorical, catColObtained],axis = 1)
print(cat_combined_data.shape)
print("\n")

#checking missing values in cat_combined_data
print(cat_combined_data.isnull().sum().to_frame().T)
#checking columns
print(catColObtained.columns)


(2919, 46)


   MSSubClass  OverallQual  OverallCond  MSZoning  Street  Alley  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinType2  Heating  HeatingQC  CentralAir  Electrical  KitchenQual  Functional  FireplaceQu  GarageType  GarageFinish  GarageQual  GarageCond  PavedDrive  PoolQC  Fence  MiscFeature  SaleType  SaleCondition
0           0            0            0         4       0      0         0            0          2          0          0             0           0           0         0           0          0         0            1            1          24          0          0           0         0         0             0             0             0        0          0           0           1            1           2            0           0             0           

In [22]:

#Filling mode values for nan in categorical columns in the existing NewTestTrainWithColDrop[num]
cat_cols = cat_combined_data.columns
for cols in cat_cols:
    cat_combined_data[cols] = cat_combined_data[cols].fillna(cat_combined_data[cols].mode)

#Checking the cat_combined data after filling
print(cat_combined_data.isnull().sum().to_frame().T)
print("\nData\n")
#print(cat_combined_data.head())
print(cat_combined_data.shape)
print("\n\n")

   MSSubClass  OverallQual  OverallCond  MSZoning  Street  Alley  LotShape  LandContour  Utilities  LotConfig  LandSlope  Neighborhood  Condition1  Condition2  BldgType  HouseStyle  RoofStyle  RoofMatl  Exterior1st  Exterior2nd  MasVnrType  ExterQual  ExterCond  Foundation  BsmtQual  BsmtCond  BsmtExposure  BsmtFinType1  BsmtFinType2  Heating  HeatingQC  CentralAir  Electrical  KitchenQual  Functional  FireplaceQu  GarageType  GarageFinish  GarageQual  GarageCond  PavedDrive  PoolQC  Fence  MiscFeature  SaleType  SaleCondition
0           0            0            0         0       0      0         0            0          0          0          0             0           0           0         0           0          0         0            0            0           0          0          0           0         0         0             0             0             0        0          0           0           0            0           0            0           0             0           0           0

In [0]:
#Hot encoding all the string type/boolean categorical variabes
categoricalFeatures =['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
catColObtained = pd.get_dummies(catColObtained, columns = categoricalFeatures)


In [24]:

#combining encoded categories with actual categoris 
encoded_cat_columns = pd.concat([ActuallyCategorical,catColObtained], axis = 1)
print(encoded_cat_columns.shape)

#encoding the numeric categories
num_cat = ['MSSubClass', 'OverallQual', 'OverallCond']
encoded_cat_combined_data = pd.get_dummies(encoded_cat_columns, columns = num_cat)

print(encoded_cat_combined_data.head())
print(encoded_cat_combined_data.shape)

(2919, 270)
     MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Grvl  Street_Pave  Alley_Grvl  Alley_NO  Alley_Pave  LotShape_IR1  LotShape_IR2  LotShape_IR3  LotShape_Reg  LandContour_Bnk  LandContour_HLS  LandContour_Low  LandContour_Lvl  Utilities_AllPub  Utilities_NoSeWa  LotConfig_Corner  LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside  LandSlope_Gtl  LandSlope_Mod  LandSlope_Sev  Neighborhood_Blmngtn  Neighborhood_Blueste  Neighborhood_BrDale  Neighborhood_BrkSide  Neighborhood_ClearCr  Neighborhood_CollgCr  Neighborhood_Crawfor  Neighborhood_Edwards  Neighborhood_Gilbert  Neighborhood_IDOTRR  Neighborhood_MeadowV  Neighborhood_Mitchel  Neighborhood_NAmes  Neighborhood_NPkVill  Neighborhood_NWAmes  Neighborhood_NoRidge  Neighborhood_NridgHt  Neighborhood_OldTown  Neighborhood_SWISU  Neighborhood_Sawyer  Neighborhood_SawyerW  Neighborhood_Somerst  Neighborhood_StoneBr  Neighborhood_Timber  Neighborhood_Veenker  Condition1_Artery  \
0

In [25]:
#combining all the numeric and categorical columns 
final_combined_data = pd.concat([num_combined_data,encoded_cat_combined_data], axis = 1)
print(final_combined_data.shape)

#checking missing values for the final combined df
pd.DataFrame(final_combined_data.isnull().sum()).T


(2919, 336)


,Id,LotFrontage,LotArea,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_NO,Alley_Pave,LotShape_IR1,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_Bnk,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_AllPub,Utilities_NoSeWa,LotConfig_Corner,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Gtl,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Artery,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Artery,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_1Fam,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Fin,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Flat,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_ClyTile,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsbShng,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsbShng,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkCmn,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,ExterQual_Ex,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Ex,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Pconc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtQual_Ex,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_NO,BsmtQual_TA,BsmtCond_Fa,BsmtCond_Gd,BsmtCond_NO,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Av,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_NO,BsmtExposure_No,BsmtFinType1_ALQ,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_NO,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_ALQ,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_NO,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,HeatingQC_Ex,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,CentralAir_N,CentralAir_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,KitchenQual_Ex,KitchenQual_Fa,KitchenQual_Gd,KitchenQual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,FireplaceQu_Ex,FireplaceQu_Fa,FireplaceQu_Gd,FireplaceQu_NO,FireplaceQu_Po,FireplaceQu_TA,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NO,GarageFinish_Fin,G

In [26]:
#splitting combined data into train and test data
X_train = final_combined_data.xs(0)
X_test = final_combined_data.xs(1)

print(X_train.head())

   Id  LotFrontage  LotArea  YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold  MSZoning_C (all)  MSZoning_FV  MSZoning_RH  MSZoning_RL  MSZoning_RM  Street_Grvl  Street_Pave  Alley_Grvl  Alley_NO  Alley_Pave  LotShape_IR1  LotShape_IR2  LotShape_IR3  LotShape_Reg  LandContour_Bnk  LandContour_HLS  LandContour_Low  LandContour_Lvl  Utilities_AllPub  Utilities_NoSeWa  LotConfig_Corner  LotConfig_CulDSac  LotConfig_FR2  LotConfig_FR3  LotConfig_Inside  LandSlope_Gtl  LandSlope_Mod  LandSlope_Sev  Neighborhood_Blmngtn  Neighborhood_Blueste  Neighborhood_BrDale  Neighborhood_BrkSide  Neighborhood_ClearCr  Neighborhood_CollgCr  Neighborhood_Crawfor  Neighborhood_Edwards  \
0   1 

In [58]:
#1A Building default decision tree regressor
clf = DecisionTreeRegressor()
clf.fit(X_train,Y_train)
clf_predict = clf.predict(X_test)

clf_cv_score = cross_val_score(clf, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*clf_cv_score.mean()))

44456.020197008744


In [28]:
#1B Building tuned decision tree regressor
#Using gridSearch
clf_g = DecisionTreeRegressor()
clf_param =  {"max_depth": range(40, 200, 10), "min_samples_leaf": range(7, 70, 7)}
clf_gridSearch = GridSearchCV(clf_g, clf_param, cv = 5)
clf_gridSearch.fit(X_train, Y_train)
clf_param = clf_gridSearch.best_params_
print(clf_param)


{'max_depth': 90, 'min_samples_leaf': 7}


In [29]:
#decision tree regressor with tuned parameters
clf_tuned = DecisionTreeRegressor(**clf_param)
clf_tuned.fit(X_train,Y_train)

clf_cv_score = cross_val_score(clf_tuned, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*clf_cv_score.mean()))

39174.59395857162


In [0]:
#prediction using decision tree regressor
resDF = pd.DataFrame(clf_tuned.predict(X_test), columns = ['SalePrice'])
#copying to csv 
dt_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
dt_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultDTR.csv", index = None)

In [31]:
#2A using default RandomForestRegressor
rfc = RandomForestRegressor()
rfc.fit(X_train, Y_train)

rfc_cv_score = cross_val_score(rfc, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*rfc_cv_score.mean()))

32325.4818544891


In [32]:
#2B using tuned RandomForestRegressor
rfc_r = RandomForestRegressor()
rfc_param = {"max_depth": range(40, 200, 10), "min_samples_leaf": range(7, 70, 7)}

rfc_GridSearch = GridSearchCV(rfc_r, rfc_param, cv = 5)
rfc_GridSearch.fit(X_train, Y_train)
rfc_param = rfc_GridSearch.best_params_
print(rfc_param)


{'max_depth': 120, 'min_samples_leaf': 7}


In [33]:
#random forest regressor with tuned parameters
rfc_tuned = RandomForestRegressor(**rfc_param)
rfc_tuned.fit(X_train,Y_train)

rfc_cv_score = cross_val_score(rfc_tuned, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*rfc_cv_score.mean()))

32476.281711929507


In [0]:
#prediction using random forest regressor
resDF = pd.DataFrame(rfc_tuned.predict(X_test), columns = ['SalePrice'])
#copying to csv 
rf_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
rf_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultRFR.csv", index = None)

In [35]:
#3A Gradient boositing regressor 
gb = GradientBoostingRegressor()
gb.fit(X_train, Y_train)

gb_cv_score = cross_val_score(gb, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*gb_cv_score.mean()))

27650.687427915534


In [36]:
#3B Gradient Boosting with tuning
"""
gb_r = GradientBoostingRegressor()
gb_param = {"max_depth": range(40, 200, 10)}
gb_randomSearch = RandomizedSearchCV(gb_r, gb_param, n_iter = 15, cv =3)

gb_randomSearch.fit(X_train, Y_train)
gb_param = gb_randomSearch.best_params_
print(gb_param)
"""


'\ngb_r = GradientBoostingRegressor()\ngb_param = {"max_depth": range(40, 200, 10)}\ngb_randomSearch = RandomizedSearchCV(gb_r, gb_param, n_iter = 15, cv =3)\n\ngb_randomSearch.fit(X_train, Y_train)\ngb_param = gb_randomSearch.best_params_\nprint(gb_param)\n'

In [0]:
"""
#Gradient boosting regressor with tuned parameters
gb_tuned = GradientBoostingRegressor(**gb_param)
gb_tuned.fit(X_train,Y_train)

gb_cv_score = cross_val_score(gb_tuned, X_train, Y_train, cv = 3, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*gb_cv_score.mean()))
"""

"\n#Gradient boosting regressor with tuned parameters\ngb_tuned = GradientBoostingRegressor(**gb_param)\ngb_tuned.fit(X_train,Y_train)\n\ngb_cv_score = cross_val_score(gb_tuned, X_train, Y_train, cv = 3, scoring = 'neg_mean_squared_error')\nprint(math.sqrt(-1*gb_cv_score.mean()))\n"

In [0]:
#since default gb regressor gave lower RMSE we use the default one
#prediction using gradient boosting regressor
resDF = pd.DataFrame(gb.predict(X_test), columns = ['SalePrice'])
#copying to csv 
gb_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
gb_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultGBR.csv", index = None)

In [38]:
"""
Normalizing the data before using mlp and support vector methods
to avoid bias in the data.
So we go back to the preprocessing of the data 
"""
#Normalization of only numeric data is needed
scaler = MinMaxScaler()
scaler.fit(num_combined_data.xs(0))
print(num_combined_data.xs(0).head())
normalized_numeric_trainData = scaler.transform(num_combined_data.xs(0))
print("after normalization")
normalized_TrainNum_df = pd.DataFrame(normalized_numeric_trainData)
print(normalized_TrainNum_df.head())

#normalizing test data as well.
normalized_numeric_test_data = scaler.transform(num_combined_data.xs(1))
normalized_TestNum_df = pd.DataFrame(normalized_numeric_test_data)

#combining normalized numeric test train
norm_numeric_DF = pd.concat([normalized_TrainNum_df, normalized_TestNum_df], keys = [0,1])

#finally combining numeric and categorical data
new_combined_data = pd.concat([norm_numeric_DF, encoded_cat_combined_data], axis = 1)

#combined data check
print("combined data")
print(new_combined_data.shape)

#splitting new combined_data once more into train and test sets
X_Ntrain = new_combined_data.xs(0)
X_Ntest = new_combined_data.xs(1)

#testing train and test data
print("Testing train and test data")
print(X_Ntrain.shape)
print(X_Ntest.shape)

   Id  LotFrontage  LotArea  YearBuilt  YearRemodAdd  MasVnrArea  BsmtFinSF1  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  1stFlrSF  2ndFlrSF  LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd  Fireplaces  GarageYrBlt  GarageCars  GarageArea  WoodDeckSF  OpenPorchSF  EnclosedPorch  3SsnPorch  ScreenPorch  PoolArea  MiscVal  MoSold  YrSold
0   1         65.0     8450       2003          2003       196.0       706.0         0.0      150.0        856.0       856       854             0       1710           1.0           0.0         2         1             3             1             8           0       2003.0         2.0       548.0           0           61              0          0            0         0        0       2    2008
1   2         80.0     9600       1976          1976         0.0       978.0         0.0      284.0       1262.0      1262         0             0       1262           0.0           1.0         2         0   

In [39]:
#3C Gradient boositing regressor on the normalized data
gb = GradientBoostingRegressor()
gb.fit(X_Ntrain, Y_train)

gb_cv_score = cross_val_score(gb, X_Ntrain, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*gb_cv_score.mean()))

27682.453644877438


In [0]:
#prediction using gradient boosting regressor
resDF = pd.DataFrame(gb.predict(X_test), columns = ['SalePrice'])
#copying to csv 
gb_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
gb_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultGBRAfterNorm.csv", index = None)

In [41]:
#4 using MLP regressor on normalized data
MLP_n = MLPRegressor()
MLP_n.fit(X_Ntrain, Y_train)

MLP_cv_score = cross_val_score(MLP_n, X_Ntrain, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*MLP_cv_score.mean()))

184648.09749901676


In [42]:
#since normalized data not good enough for mlp
#we stick to mlp on the original data
MLP = MLPRegressor()
MLP.fit(X_train, Y_train)

MLP_cv_score = cross_val_score(MLP, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*MLP_cv_score.mean()))

46434.125439040225


In [0]:
#prediction using MLP regressor
resDF = pd.DataFrame(MLP.predict(X_test), columns = ['SalePrice'])
#copying to csv 
MLPR_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
MLPR_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultMLPR.csv", index = None)

In [45]:
#5 using gradient descent regressor on normalized data
sgd = SGDRegressor(max_iter = 1000, tol = 1e-3)
sgd.fit(X_Ntrain, Y_train)

sgd_cv_score = cross_val_score(sgd, X_Ntrain, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*sgd_cv_score.mean()))


30830.941206745425


In [0]:
#prediction using gradient descent regressor
resDF = pd.DataFrame(sgd.predict(X_Ntest), columns = ['SalePrice'])
#copying to csv 
sgd_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
sgd_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultSGD.csv", index = None)

In [47]:
#6 using support vector regressor on the normalized data
svr = SVR()
svr.fit(X_Ntrain, Y_train)

svr_cv_score = cross_val_score(svr, X_Ntrain, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*svr_cv_score.mean()))


81433.3092208208


In [0]:
#prediction using support vector regressor
resDF = pd.DataFrame(svr.predict(X_Ntest), columns = ['SalePrice'])
#copying to csv 
svr_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
svr_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultSVR.csv", index = None)


In [0]:
#

In [0]:
"""
From kaggle the best scores are obtained for below 3 models:
1. Gradient Boosting regressor
2. Tuned Random Forest Regressor
3. Gradient Descent Regressor
"""

In [49]:
#stacking all the models over normalized data
models = [gb, rfc_tuned, sgd]

S_train, S_test = stacking(models, 
                           X_Ntrain, Y_train, X_Ntest, 
                           regression = True,
                           mode = 'oof_pred_bag',
                           needs_proba = False,
                           save_dir = None,
                           n_folds = 4,
                           verbose = 2)


task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingRegressor]
    fold  0:  [16224.16691976]
    fold  1:  [18609.05937445]
    fold  2:  [15820.88663563]
    fold  3:  [16401.01512924]
    ----
    MEAN:     [16763.78201477] + [1085.91948299]
    FULL:     [16763.78201477]

model  1:     [RandomForestRegressor]
    fold  0:  [19668.18477595]
    fold  1:  [22172.94236841]
    fold  2:  [18384.94175011]
    fold  3:  [19163.82435671]
    ----
    MEAN:     [19847.47331279] + [1418.30225755]
    FULL:     [19847.47331279]

model  2:     [SGDRegressor]
    fold  0:  [17882.91265258]
    fold  1:  [20151.30420960]
    fold  2:  [17376.64891249]
    fold  3:  [18548.24256705]
    ----
    MEAN:     [18489.77708543] + [1045.39817368]
    FULL:     [18489.77708543]



In [50]:
#1 gradient boosting over the stack
gb_stack = GradientBoostingRegressor()
gb_stack.fit(S_train, Y_train)

gb_stack_cv_score = cross_val_score(gb_stack, X_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*gb_cv_score.mean()))

27682.453644877438


In [0]:
#prediction using stacking model
resDF = pd.DataFrame(gb_stack.predict(S_test), columns = ['SalePrice'])
#copying to csv  
stackModel_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
stackModel_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultGBStack.csv", index = None)


In [52]:
#2 Random Forest Regressor over stack
rfR_stack = RandomForestRegressor()
rfR_stack.fit(S_train, Y_train)

rfR_stack_cv_score = cross_val_score(rfR_stack, S_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*rfR_stack_cv_score.mean()))

33313.3920155379


In [0]:
#prediction using stacking model
resDF = pd.DataFrame(rfR_stack.predict(S_test), columns = ['SalePrice'])
#copying to csv  
stackModel_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
stackModel_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultRFStack.csv", index = None)


In [54]:
#2 Decision Tree Regressor over stack
dt_stack = DecisionTreeRegressor()
dt_stack.fit(S_train, Y_train)

dt_stack_cv_score = cross_val_score(dt_stack, S_train, Y_train, cv = 10, scoring = 'neg_mean_squared_error')
print(math.sqrt(-1*dt_stack_cv_score.mean()))

38982.48810493463


In [0]:
#prediction using stacking model
resDF = pd.DataFrame(dt_stack.predict(S_test), columns = ['SalePrice'])
#copying to csv  
stackModel_resultDF = pd.concat([X_test['Id'], resDF], axis = 1)
stackModel_resultDF.to_csv("/gdrive/My Drive/Assignments/TeamAssignment1/resultDTStack.csv", index = None)
